In [1]:
from tqdm import tqdm
import pandas as pd
pd.set_option("display.precision",16 )
train_path='../data/underexpose_train/'
test_path='../data/underexpose_test/'
now_p=6

col_name=['item_id']
for i in range(0,128):
    col_name.append('tv'+str(i))
for i in range(0,128):
    col_name.append('iv'+str(i))



itemft=pd.read_csv(train_path+'underexpose_item_feat.csv',low_memory=False,names=col_name)
itemft=itemft.replace('[\[\]]','',regex=True)#regex=True is the key
#underexpose_item_feat.apply(lambda x: x.str.replace('[\[\]]',''),axis=1)
userft=pd.read_csv(train_path+'underexpose_user_feat.csv',low_memory=False,names=['user_id','age','gender','city'],dtype=\
                  {'user_id': np.int32,'age':'category','gender':'category','city':'category'})


In [81]:
for p in tqdm(range(0,now_p+1)):


    tnc=pd.read_csv(train_path+r'underexpose_train_click-{}.csv'.format(p),low_memory=False,names=['user_id','item_id','time'])
    ttc=pd.read_csv(test_path+r'underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(p,p),low_memory=False\
                   ,names=['user_id','item_id','time'])

    # ttt=pd.read_csv(test_path+'utc_click-{}.csv'.format(p),low_memory=False)
    # ttt.drop(ttt.columns[0],axis=1,inplace=True) # !!
    a=userft.user_id.value_counts()
    dup_user_id=a[a>1].index.to_list()
    dup_index=userft.loc[userft.user_id.isin(dup_user_id),:].index.to_list()
    dup_index=[i for i in dup_index if i%2 == 0]
    userft.drop(dup_index,inplace=True)
    temp=tnc.merge(itemft,how='left')
    assert tnc.shape[0]==temp.shape[0]

    all_tn=temp.merge(userft,left_on=['user_id'],right_on=['user_id'],how='left')
    assert temp.shape[0] == all_tn.shape[0]
    all_tn.to_csv(train_path+'train_{}.csv'.format(p),index=False)

100%|██████████| 7/7 [24:43<00:00, 211.94s/it]


## get matrix

In [2]:
from tqdm import tqdm
import pandas as pd
train_path='../data/underexpose_train/'
test_path='../data/underexpose_test/'
now_p=6
df=pd.read_csv(train_path+'train_{}.csv'.format(now_p),low_memory=False)

In [4]:
item_user_matrix=df.pivot_table(index='item_id',columns='user_id',aggfunc='size',fill_value=0)

In [5]:
item_user_matrix.shape

(48662, 18575)

In [11]:
item_user_matrix_min=item_user_matrix.dropna()

In [9]:
item_user_matrix.to_hdf(train_path+'item_user_matrix.h5',key='result',index=False)

HDF5ExtError: Problems creating the Array.

In [14]:
df.shape

(310372, 262)

In [15]:
df.dropna().shape

(71688, 262)

## get non-null database

In [16]:
df_min=df.dropna()

In [17]:
df_min.to_csv(train_path+'train_min_{}'.format(now_p),index=False)

In [4]:
df_min=pd.read_csv(train_path+'train_min_{}'.format(now_p))

In [5]:
database=df_min.copy().drop(['user_id','item_id'],axis=1)
database=database.replace({'M':1,'F':0})
database=database.astype({'age': 'int','gender':'int','city':'int'})
database.head(3)

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9841110719125040,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,4
1,0.9842627512424618,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,2
2,0.9840694762116832,5.0535550117492685,1.2165696620941162,0.3485578894615173,0.3808794021606445,5.7227177619934073,-0.9034663438796996,-2.9302191734313965,-2.0493977069854736,-7.3847928047180176,...,-1.3566752672195437,-1.0602157115936279,0.3031310141086578,1.5827329158782959,1.4230424165725708,2.8760449886322017,-0.5718842148780823,3,0,5


## true value

In [7]:
c=6
temp=pd.read_csv(test_path + '/underexpose_test_click-{}/underexpose_test_click-{}.csv'.format(c,c), header=None,  names=['user_id', 'item_id', 'time'])
temp=temp.sort_values(['user_id','time'],ascending=[True,False]).groupby(['user_id']).agg('max').reset_index()
query=temp.merge(itemft,how='left').merge(userft,how='left').drop(['user_id','item_id'],axis=1)
query=query.replace({'M':1,'F':0})
query=query.astype({'age': 'int','gender':'int','city':'int'},errors='ignore')
query.head(3)

query.info()
query.shape
query.select_dtypes(['category'])
query['age'].dtype =='category'

database['age'].median()

summary=database.mean()
for col in query.isna().any().loc[query.isna().any() == True].index:
    if col in ['age','gender','city']:
        a=summary[col]
    else:
        a=database[col].median()
    query[col]=query[col].replace(np.nan,a)
query.shape == query.dropna().shape

query.head()
database.head()

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
query1=ss.fit_transform(query)
database1=ss.fit_transform(database)

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9842771241305298,2.101147174835205,-0.8465766906738281,0.0573386922478676,-3.1112377643585205,-0.8484025001525879,2.9340057373046875,1.1302022933959959,-0.5584614276885986,-2.9435551166534424,...,1.5269055366516111,-2.7677159309387207,-0.5956848263740540,0.1572125703096390,1.0126974582672121,2.7917819023132324,0.3088803291320801,8,1.0,4
1,0.9842517769474944,1.900734305381775,2.1579475402832031,3.0377004146575928,-3.0554065704345703,1.2677739858627319,-1.0928924083709717,-3.7414407730102539,-1.1887992620468140,-3.2973101139068604,...,1.0075087547302246,-1.0377058982849119,0.3403505682945251,2.2763986587524414,2.3016302585601807,1.0346159934997561,-0.8432886004447937,NaN,NaN,NaN
2,0.9842692676835492,1.830065131187439,1.8811055421829224,2.6448986530303955,-2.4803488254547119,-1.0310925245285034,1.9788519144058228,-1.2319024801254272,-1.8476966619491575,-3.5557911396026611,...,-0.2710131108760834,0.2487333118915558,2.4548890590667725,-0.1799300462007523,2.5021350383758545,0.2804744839668274,0.44003647565841675,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1821 entries, 0 to 1820
Columns: 260 entries, time to city
dtypes: category(2), float64(254), object(4)
memory usage: 3.6+ MB


(1821, 260)

,age,city
0,8,4
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,6,1
...,...,...
1816,NaN,NaN
1817,NaN,NaN
1818,NaN,NaN
1819,NaN,NaN


True

4.0

True

,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9842771241305298,2.101147174835205,-0.8465766906738281,0.0573386922478676,-3.1112377643585205,-0.8484025001525879,2.9340057373046875,1.1302022933959959,-0.5584614276885986,-2.9435551166534424,...,1.5269055366516111,-2.7677159309387207,-0.5956848263740540,0.1572125703096390,1.0126974582672121,2.7917819023132324,0.3088803291320801,8,1.0000000000000000,4
1,0.9842517769474944,1.900734305381775,2.1579475402832031,3.0377004146575928,-3.0554065704345703,1.2677739858627319,-1.0928924083709717,-3.7414407730102539,-1.1887992620468140,-3.2973101139068604,...,1.0075087547302246,-1.0377058982849119,0.3403505682945251,2.2763986587524414,2.3016302585601807,1.0346159934997561,-0.8432886004447937,4.248995647807164,0.1149425287356322,3.674087713424841
2,0.9842692676835492,1.830065131187439,1.8811055421829224,2.6448986530303955,-2.4803488254547119,-1.0310925245285034,1.9788519144058228,-1.2319024801254272,-1.8476966619491575,-3.5557911396026611,...,-0.2710131108760834,0.2487333118915558,2.4548890590667725,-0.1799300462007523,2.5021350383758545,0.2804744839668274,0.44003647565841675,4.248995647807164,0.1149425287356322,3.674087713424841
3,0.9842643606374868,1.8137321472167969,1.3914031982421875,2.1369509696960449,-4.1689491271972665,-2.8606204986572266,1.9786089658737185,-0.2026271671056747,-1.7621158361434937,-3.4887967109680176,...,-1.4988899230957031,1.6837480068206787,1.1026927232742310,-0.2537772655487061,2.9432177543640137,1.3797429800033569,-0.885633647441864,4.248995647807164,0.1149425287356322,3.674087713424841
4,0.9842105925855824,0.8316391706466675,1.8352080583572388,1.0361447334289549,-3.9268870353698730,-0.4585402011871338,0.8195793628692627,3.3453412055969238,-4.2287759780883789,-3.3158850669860840,...,1.4824553728103638,4.6624727249145508,0.7338652014732361,1.7606782913208008,1.0468974113464355,0.5324596166610718,-0.70954829454422,6,1.0000000000000000,1


,time,tv0,tv1,tv2,tv3,tv4,tv5,tv6,tv7,tv8,...,iv121,iv122,iv123,iv124,iv125,iv126,iv127,age,gender,city
0,0.9841110719125040,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,4
1,0.9842627512424618,4.1481380462646484,-0.2139974832534789,-0.2748044133186340,0.3943611383438110,-2.2072267532348642,-1.4376068115234375,2.0980615615844727,-0.6124892234802246,-4.8264927864074716,...,-2.9370822906494136,-0.9811164140701294,0.4257615506649017,1.4910547733306885,3.8896303176879878,1.4391320943832395,-0.0809062123298645,4,0,2
2,0.9840694762116832,5.0535550117492685,1.2165696620941162,0.3485578894615173,0.3808794021606445,5.7227177619934073,-0.9034663438796996,-2.9302191734313965,-2.0493977069854736,-7.3847928047180176,...,-1.3566752672195437,-1.0602157115936279,0.3031310141086578,1.5827329158782959,1.4230424165725708,2.8760449886322017,-0.5718842148780823,3,0,5
3,0.9842570189950302,2.9716651439666748,-0.5788695812225342,2.2469174861907959,-1.7302743196487431,-2.3264360427856445,-0.5912094712257385,-2.3438596725463867,0.4770196080207825,-3.7080161571502690,...,0.9559630155563354,-0.0576305389404297,-2.8185224533081050,-3.0371842384338379,0.8351709246635437,1.1768972873687744,0.4194931983947754,2,0,5
4,0.9840924399002436,0.4042505621910095,-1.0307354927062988,-0.3946554064750672,-1.1291629076004028,1.5390796661376951,0.4129871129989624,-0.1573128253221512,0.0748407095670700,-3.0261974334716801,...,-1.2563368082046509,-1.1090230941772459,1.2311370372772217,1.1668357849121094,3.2568092346191406,2.9602167606353764,-1.0459980964660645,4,0,6


## query records

In [183]:
temp=temp.sort_values(by=['user_id','time'],ascending=[True,False]).groupby(['user_id']).apply(\
                                                        lambda x : x.nlargest(2,'time').min()).drop(['user_id'],axis=1).reset_index()
temp.head(3) # GET the second last click

query=temp.merge(itemft,how='left').merge(userft,how='left').drop(['user_id','item_id'],axis=1)
query=query.replace({'M':1,'F':0})
query=query.astype({'age': 'int','gender':'int','city':'int'},errors='ignore')
query.head(3)

query.info()
query.shape
query.select_dtypes(['category'])
query['age'].dtype =='category'

database['age'].median()

summary=database.mean()
for col in query.isna().any().loc[query.isna().any() == True].index:
    if col in ['age','gender','city']:
        a=summary[col]
    else:
        a=database[col].median()
    query[col]=query[col].replace(np.nan,a)
query.shape == query.dropna().shape

query.head()
database.head()

from sklearn.preprocessing import StandardScaler
ss=StandardScaler()
query1=ss.fit_transform(query)
database1=ss.fit_transform(database)

## find similarity

In [8]:
import faiss

database1.dtype
database1=np.ascontiguousarray(database1.astype('float32'))
query1=np.ascontiguousarray(query.astype('float32'))



dim=260
k=50 # recall 10
nlist=100
quantizer = faiss.IndexFlatL2(dim)
idx=faiss.IndexIVFPQ(quantizer,dim,nlist,10,8)
idx.nprobe =10

# index_sim.add(np.ascontiguousarray(tr_features_np))


idx.train(np.ascontiguousarray(database1))
idx.add(np.ascontiguousarray(database1))
d,i=idx.search(np.ascontiguousarray(query1),100)

dtype('float64')

In [9]:
d.shape

(1821, 100)

In [10]:
result=np.ndarray(shape=d.shape)

In [11]:
temp.user_id.to_numpy()

array([   17,    28,    39, ..., 35415, 35426, 35437])

In [12]:
for row_i in tqdm(range(len(d))):
    result[row_i]=df_min.loc[i[row_i],'item_id'].values

100%|██████████| 1821/1821 [00:00<00:00, 2763.47it/s]


In [13]:
temp.user_id.to_numpy().reshape(-1,1).shape
result.shape

(1821, 1)

(1821, 100)

In [14]:
result=np.concatenate((temp.user_id.to_numpy().reshape(-1,1),result),axis=1).astype('int')

In [15]:

resultdf=pd.DataFrame(result,columns=['col'+ str(i) for i in range(0,101)]).set_index('col0')

In [17]:
sm=0
for top100 in list(map(set,resultdf.values)):
    if len(top100)<50:
        sm+=1
sm

538

In [ ]:
top50_click = whole_click['item_id'].value_counts().index[:50].values

In [296]:
for item in list(map(set,resultdf.values)):
    item
    if len(item)< 50:

{143,
 206,
 932,
 1131,
 1399,
 2143,
 3113,
 3648,
 5192,
 5877,
 5878,
 5946,
 6151,
 6338,
 6918,
 7245,
 7792,
 7936,
 8724,
 9415,
 10956,
 12120,
 16166,
 16258,
 20170,
 22391,
 25309,
 26704,
 26992,
 36285,
 38168,
 39404,
 39660,
 41717,
 45804,
 48347,
 48592,
 54193,
 56692,
 62779,
 66991,
 71189,
 73937,
 74052,
 89548,
 95499,
 111937}

{2617,
 3161,
 4275,
 16258,
 17724,
 22139,
 29714,
 32140,
 34415,
 34587,
 35568,
 39626,
 39706,
 41829,
 42468,
 42615,
 45360,
 48053,
 50696,
 61675,
 62001,
 62779,
 67247,
 80160,
 84108,
 88774,
 88851,
 100150,
 104470,
 107623,
 108361,
 108600,
 109722,
 113378,
 114373,
 114634,
 116890}

{15964,
 17995,
 20829,
 27206,
 27424,
 31873,
 34270,
 38732,
 39371,
 40795,
 41838,
 42965,
 47846,
 48544,
 48611,
 52067,
 52521,
 53344,
 54940,
 55632,
 61638,
 62741,
 63485,
 64920,
 66040,
 67862,
 68790,
 69305,
 72463,
 72604,
 73294,
 73361,
 75937,
 76181,
 76370,
 76921,
 83166,
 83484,
 84091,
 84134,
 85318,
 85759,
 86329,
 87030,
 87218,
 87368,
 87848,
 89969,
 90581,
 91756,
 92905,
 94837,
 96407,
 100371,
 100460,
 100977,
 104127,
 104714,
 104782,
 107269}

{4516,
 21255,
 26340,
 30087,
 36247,
 49594,
 52411,
 58462,
 58765,
 59036,
 59323,
 60079,
 63579,
 65975,
 66954,
 69916,
 71372,
 71594,
 72395,
 73685,
 73887,
 74282,
 78070,
 78307,
 81772,
 82616,
 82648,
 84122,
 85964,
 86045,
 87042,
 89148,
 89618,
 92234,
 93601,
 97395,
 98438,
 98815,
 99054,
 99373,
 101157,
 101211,
 101402,
 102045,
 102659,
 103328,
 104205,
 104456,
 104632,
 105002,
 106351,
 106683,
 107107,
 107444,
 108224,
 108401,
 109037,
 109934,
 110021,
 110834,
 111878,
 112211,
 113732}

{98,
 869,
 10983,
 15965,
 16307,
 20883,
 21493,
 21778,
 23667,
 23872,
 24019,
 24053,
 25208,
 28487,
 35917,
 39387,
 42201,
 45349,
 46917,
 49071,
 49105,
 51041,
 51304,
 54275,
 57690,
 60060,
 63487,
 65835,
 65840,
 65852,
 66424,
 66854,
 66855,
 66857,
 67896,
 68214,
 68355,
 69517,
 69746,
 71779,
 73828,
 76273,
 88954,
 89413,
 90516,
 91602,
 91604,
 93674,
 94075,
 96590,
 96593,
 96944,
 97014,
 97199,
 98772,
 98952,
 100504,
 101380,
 103128,
 105510,
 105998,
 107578,
 107643,
 110104,
 110935,
 112580}

...

{8267,
 8997,
 17711,
 31693,
 32681,
 33759,
 36620,
 36712,
 37189,
 39238,
 39371,
 39493,
 41775,
 42771,
 42965,
 43113,
 45497,
 45833,
 46526,
 46782,
 47308,
 47414,
 47846,
 49625,
 51109,
 52802,
 53608,
 53770,
 54027,
 55480,
 55992,
 63326,
 63485,
 64858,
 68559,
 69630,
 71695,
 73264,
 77127,
 81222,
 88041,
 88077,
 88686,
 90761,
 93444,
 94585,
 94836,
 94981,
 96352,
 97522,
 97873,
 98123,
 98287,
 98839,
 103952,
 106525,
 107623,
 109689,
 109955,
 110081,
 110282,
 110499,
 110584,
 112739,
 116271}

{556,
 11048,
 13894,
 14103,
 14515,
 14529,
 14822,
 20777,
 24139,
 28912,
 29319,
 29523,
 30601,
 30967,
 31115,
 31184,
 32935,
 33851,
 37158,
 37950,
 39560,
 41333,
 44302,
 44403,
 53747,
 53860,
 54256,
 55183,
 55400,
 55659,
 56858,
 58162,
 64377,
 68034,
 69513,
 76040,
 80393,
 82078,
 84804,
 86194,
 92407,
 93243,
 94063,
 97992,
 98056,
 98686,
 99600,
 101289,
 102598,
 104071,
 105884,
 106036,
 107918,
 110067,
 110319,
 114012}

{9365,
 20915,
 32423,
 33041,
 33636,
 36652,
 38291,
 39855,
 40859,
 41607,
 42362,
 43395,
 46806,
 48603,
 50415,
 51780,
 52325,
 55507,
 56458,
 56462,
 57948,
 58243,
 58895,
 59181,
 59925,
 60044,
 61778,
 61945,
 63130,
 64708,
 65132,
 65612,
 66666,
 74972,
 75390,
 75605,
 76717,
 76919,
 78981,
 94156,
 99617,
 103882,
 103890,
 105207,
 105257,
 105289,
 105492,
 105973,
 106753,
 107709,
 108300,
 110086,
 111023,
 111558,
 111927,
 111983,
 112646,
 115202,
 115357,
 116534,
 116592}

{132,
 4220,
 7940,
 8012,
 8463,
 9070,
 12600,
 13408,
 15793,
 17894,
 18831,
 19888,
 22412,
 29052,
 34134,
 37152,
 37250,
 38208,
 39066,
 39353,
 40960,
 43335,
 44241,
 45580,
 46552,
 47896,
 49673,
 50805,
 58138,
 59510,
 64850,
 65795,
 67978,
 69623,
 71072,
 72269,
 79743,
 81697,
 86510,
 86606,
 88167,
 88229,
 92454,
 94065,
 95744,
 98272,
 110098,
 110727,
 110773,
 111464}

KeyboardInterrupt: 

In [266]:
np.savetxt('result.csv',result,delimiter=',',fmt='%i')